In [2]:
import os
import requests
import tweepy as tw
import pandas as pd
from datetime import date

In [93]:
# Query earthquake with magnatuide >= 5 and alert level orange (yellow, orange, red)
URL = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=5&alertlevel=orange"
response = requests.get(URL)
earthquakes = response.json()['features']
print(len(earthquakes), "earthquake in past 30 days.")

1 earthquake in past 30 days.


In [96]:
earthquakes

[{'geometry': {'coordinates': [47.5577, 37.8075, 10], 'type': 'Point'},
  'id': 'us600068w0',
  'properties': {'alert': 'orange',
   'cdi': 7.9,
   'code': '600068w0',
   'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us600068w0&format=geojson',
   'dmin': 2.744,
   'felt': 623,
   'gap': 20,
   'ids': ',us600068w0,',
   'mag': 5.9,
   'magType': 'mww',
   'mmi': 7.785,
   'net': 'us',
   'nst': None,
   'place': '57km NE of Hashtrud, Iran',
   'rms': 0.63,
   'sig': 1492,
   'sources': ',us,',
   'status': 'reviewed',
   'time': 1573166825428,
   'title': 'M 5.9 - 57km NE of Hashtrud, Iran',
   'tsunami': 0,
   'type': 'earthquake',
   'types': ',dyfi,general-text,geoserve,ground-failure,impact-text,losspager,losspager-admin,moment-tensor,origin,phase-data,shakemap,',
   'tz': 210,
   'updated': 1573529037875,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us600068w0'},
  'type': 'Feature'}]

In [97]:
def get_tweet(keys, search_words, since_date=str(date.today()), max_item=500, lang="en", geocode=None):
    consumer_key, consumer_secret, access_token, access_token_secret = keys
    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tw.API(auth, wait_on_rate_limit=True)
    
    # Define the search term and date
    search_words = search_words + " -filter:retweets"

    # Collect tweets
    tweets = tw.Cursor(api.search, q=search_words, lang=lang, geocode=geocode, since=today).items(max_item)

    users_locs = [[tweet.user.screen_name, tweet.user.location,
                   tweet.created_at, tweet.coordinates, tweet.text] for tweet in tweets]
    tweet_text = pd.DataFrame(data=users_locs, columns=['user', "location", "date", "coordinates", "content"])
    
    return tweet_text

In [113]:
consumer_key = 'vi360ysd3PGoNN0ZrpexlhMGi'
consumer_secret = 'ZRQLsBhRa8FLfOT2OfpZoKZucHcEKvcj9OWrWIsDAa4i575T78'
access_token = '1164687316713009152-KRzfNTvtWTwANW13TsHNv2wsLitbZZ'
access_token_secret = 'YFwLyzwmIClv0VhOPko8i6Adss8macOzFOIBF8l6iKKSs'

keys = [consumer_key, consumer_secret, access_token, access_token_secret]
earthquake_tweet = {}

for i in range(len(earthquakes)):
    location = earthquakes[i]["properties"]["place"].split()[-1] # get earthquake location
    search_words = location + "+earthquake"
    tweet_text = get_tweet(keys, search_words)
    earthquake_tweet[location] = tweet_text

In [117]:
earthquake_tweet["Iran"]

,user,location,date,coordinates,content
0,opalessense,Artist Galerie Paul Janssen,2019-11-18 06:26:17,None,"Magnitude 5.9 Earthquake - Iran - Nov. 8, 2019..."
1,zachsdorfman,"""A Born Spy""--Richard Nixon",2019-11-18 06:26:00,None,This is an earthquake. \n\nhttps://t.co/0E1zwn...
2,livewxradar,USA,2019-11-18 03:22:00,None,"4.0 Earthquake 41 mi (66 km) WNW of Herowabad,..."
3,Earthquakes0,World,2019-11-18 02:44:08,None,2 hrs ago #earthquake 4.0 has hit East Azerbai...
4,geotecniaonline,Ciberespacio,2019-11-18 01:01:17,None,EQAlerts: #Earthquake M4.0 66km WNW of Herowab...
5,quakenotices,California,2019-11-18 01:01:07,None,#earthquake 2019-11-18 00:38:41 (M4.0) northw...
6,earthshook,,2019-11-18 00:58:38,"{'type': 'Point', 'coordinates': [47.7997, 37....",Mb 4 earthquake (reviewed) occured at 2019-11-...
7,EMSC,"Bruyères-Le-Chatel, France",2019-11-18 00:57:25,None,#Earthquake (#زلزله) M4.0 strikes 70 km SW of ...
8,myearthquakeapp,Earth,2019-11-18 00:57:14,None,4.0 earthquake occurred near northwestern Iran...
